In [1]:
from astropy.io import fits
import numpy as np
import os.path
import matplotlib.pyplot as plt
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import matplotlib.gridspec as gridspec
import aplpy
import pandas as pd
from photutils import SkyCircularAperture
from photutils import aperture_photometry

In [2]:
#create short table for extra positions outside chandra catalogues
ncat=7
id_temp = [1, 2, 3, 4, 5, 6, 7] #note: 8 corresponds to 101
ra_temp = [53.1280, 53.1716, 53.0212, 53.0647, 53.2462, 53.1925, 53.1064]
dec_temp = [-27.8594, -27.8208, -27.7824, -27.8626, -27.8618, -27.8668, -27.6977]
id2 = np.zeros(ncat)
ra2 = np.zeros(ncat)
dec2 = np.zeros(ncat)
for i in range(len(ra_temp)):
    id2[i] = id_temp[i]
    ra2[i] = ra_temp[i]
    dec2[i] = dec_temp[i]
cols = fits.ColDefs([fits.Column(name='ID', format='I', array=id2),
                     fits.Column(name='RA', format='D', array=ra2),
                     fits.Column(name='DEC', format='D', array=dec2)])
hdu2 = fits.BinTableHDU.from_columns(cols)
os.system('rm -f ../data/catalogues/chandra_7ms/short_table.fits')
hdu2.writeto('/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/short_table.fits')

In [3]:
#5, 6 and 7 have no visual detection whatsoever: photometry useless for them
# only 3 has usable spitzer image (3.0 aperture), rest upper limit
#1,2,3 H band reference, 4 can be K or H
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/short_table.fits'
cat = fits.open(path)[1].data
a = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105_', 'candels_125_', 'candels_160_', 'hugs_', 'spitzer_ch1_', 'spitzer_ch2_']
b = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105', 'candels_125', 'candels_160', 'hugs', 'spitzer_ch1', 'spitzer_ch2']
c = ['goods', 'goods', 'goods', 'goods', 'candels', 'candels', 'candels', 'hugs', 'spitzer', 'spitzer']
minarea = np.zeros(ncat)
visual = np.zeros(ncat)
aperture = np.zeros(ncat)
spitzer = np.zeros(ncat)
minarea[0], minarea[1], minarea[2], minarea[3] = 30,20,50,20
aperture[0], aperture[1], aperture[2], aperture[3] = 1.2,0.8,1.6,1.6
spitzer[0], spitzer[1], spitzer[2], spitzer[3] = 13.,13.,3.,13.
visual[0], visual[1], visual[2], visual[3] = 5,5,5,5
aper_pixel = 1.
min_area = 1.
for j in range(len(a)):
    for i in [0,1,2,3]:
        if j < 4:
            aper_pixel = aperture[i] / 0.03
            min_area = minarea[i]
            n_thresh = 64.
            min_cont = 0.0001
        elif j < 7:
            aper_pixel = aperture[i] / 0.06
            min_area = minarea[i]
            n_thresh = 64.
            min_cont = 0.0001
        elif j == 7:
            aper_pixel = aperture[i] * 9.4
            min_area = minarea[i]/2.
            n_thresh = 64.
            min_cont = 0.0001
        else:
            if spitzer[i] > 10.:
                aper_pixel = 0.
            else:
                aper_pixel = spitzer[i] *1.7
            min_area = 1.
            n_thresh = 64.
            min_cont = 0.0001
        if aper_pixel != 0.:
            conf_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/' + c[j] + '/' + b[j] + '.conf'
            im_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + '.fits'
            dualim_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + '.fits,/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + '.fits'
            cat_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + 'n' + str(i+1) + '.cat'
            wht_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + 'wht.fits'
            rms_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + 'rms.fits'
            rmswht_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + 'rms.fits,/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + 'wht.fits'
            check_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + 'n' + str(i+1) + '_aper.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + 'n' + str(i+1) + '_bkg.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + 'n' + str(i+1) + '_bkg_rms.fits' 
            if j < 4:
                path = 'sex -c' + conf_path + im_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE' + rms_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 90 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE'
            elif j < 7:
                path = 'sex -c' + conf_path + im_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE' + rms_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 60 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE'
            elif j == 7:
                path = 'sex -c' + conf_path + im_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE' + rms_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 50 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE'
            else:
                path = 'sex -c' + conf_path + dualim_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS,MAP_WEIGHT -WEIGHT_GAIN N,Y -WEIGHT_IMAGE' + rmswht_path + ' -DETECT_MINAREA ' + str(min_area) + ' -PHOT_APERTURES ' + str(aper_pixel) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 17 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -MASK_TYPE NONE -FILTER N'
            os.system(path)

In [4]:
flux=[]
fluxerr=[]
fluxra=[]
fluxdec=[]
usable=[]
arcsec_radius=1.
ra = cat.field('RA')
dec = cat.field('DEC')
for i in range(len(a)):
    flux.append(np.zeros(ncat))
    fluxerr.append(np.zeros(ncat))
    fluxra.append(np.zeros(ncat))
    fluxdec.append(np.zeros(ncat))
    usable.append(np.ones(ncat))
for i in [0,1,2,3]: #H band standard
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/candels/cat_candels_160_' + 'n' + str(i+1) + '.cat'
    t1 = pd.read_csv(path, delim_whitespace=True, header=None, names=['#', 'x', 'y', 'RA', 'Dec', 'MagAuto', 'MagErrAuto', 'FluxAuto', 'FluxErrAuto', 'MagAper', 'MagErrAper', 'FluxAper', 'FluxErrAper'])
    radius = ((t1['RA']-ra[i]).pow(2) + (t1['Dec']-dec[i]).pow(2)).pow(0.5)
    for j in range(len(radius)):
        if (radius[j] == min(radius)):
            flux[6][i] = t1['FluxAper'][j]
            fluxerr[6][i] = t1['FluxErrAper'][j]
            ra[i] = t1['RA'][j]
            dec[i] = t1['Dec'][j]
            fluxra[6][i] = t1['RA'][j]
            fluxdec[6][i] = t1['Dec'][j]

for j in range(len(a)): #all bands
    for i in [0,1,2,3]:
        if ((j > 7) & (i != 2)) | (j==6):
            pass
        else:
            try:
                path = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + 'n' + str(i+1) + '.cat'
                t5 = pd.read_csv(path, delim_whitespace=True, header=None, names=['#', 'x', 'y', 'RA', 'Dec', 'MagAuto', 'MagErrAuto', 'FluxAuto', 'FluxErrAuto', 'MagAper', 'MagErrAper', 'FluxAper', 'FluxErrAper'])
                snratio = t5['FluxAper']/t5['FluxErrAper']
                radius = ((t5['RA']-ra[i]).pow(2) + (t5['Dec']-dec[i]).pow(2)).pow(0.5)
                n=-1
                if j < 8:
                    arcsec_radius=0.2
                if j > 7:
                    arcsec_radius=1.
                for k in range(len(radius)):
                    if (radius[k] < arcsec_radius/3600.) & (snratio[k] > 1.):
                        if n == -1:
                            n = k
                        else:
                            if radius[k] < radius[n]:
                                n = k
                if n != -1:
                    flux[j][i] = t5['FluxAper'][n]
                    fluxerr[j][i] = t5['FluxErrAper'][n]
                    fluxra[j][i] = t5['RA'][n]
                    fluxdec[j][i] = t5['Dec'][n]
            except:
                pass
                
    

In [5]:
for j in range(len(a)):
    for i in [0,1,2,3]:  
        if flux[j][i] == 0:
            path = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + 'n' + str(i+1) + '.fits'
            im = fits.open(path)[0]
            position = SkyCoord(ra[i]*u.deg, dec[i]*u.deg, frame='icrs')
            rad=aperture[i]/2.
            if j > 7:
                if spitzer[i]>10.:
                    rad=(spitzer[i]-10.)/2.
                else:    
                    rad=spitzer[i]/2.
            circaper = SkyCircularAperture(position, r=rad*u.arcsec)
            phot_table = aperture_photometry(im, circaper)
            if phot_table['aperture_sum'][0] < 0:
                print('negative flux ' + str(phot_table['aperture_sum'][0]) + ' for source n' + str(i+1) + ' with filter ' + b[j])
            fluxerr[j][i] = phot_table['aperture_sum'][0]
#convert to Jansky
fluxratio = [1.95338693e-7,9.238062e-8,1.98796897e-7,4.12286699e-7,1.12851106e-7,1.16912375e-7,1.51876003e-7,9.9999999e-10,3.92276e-6,4.80825e-6]
for j in range(len(fluxratio)):
    flux[j] = flux[j]*fluxratio[j]
    fluxerr[j] = fluxerr[j]*fluxratio[j]

In [6]:

flux_cat = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for i in [0,1,2,3]:
    flux_cat[0].append(2001 + i)
    flux_cat[1].append(flux[0][i])
    flux_cat[2].append(fluxerr[0][i])
    flux_cat[3].append(flux[1][i])
    flux_cat[4].append(fluxerr[1][i])
    flux_cat[5].append(flux[2][i])
    flux_cat[6].append(fluxerr[2][i])
    flux_cat[7].append(flux[3][i])
    flux_cat[8].append(fluxerr[3][i])
    if (i==0):
        flux_cat[9].append(-99)
        flux_cat[10].append(0)
    else:
        flux_cat[9].append(flux[4][i])
        flux_cat[10].append(fluxerr[4][i])
    flux_cat[11].append(flux[5][i])
    flux_cat[12].append(fluxerr[5][i])
    flux_cat[13].append(flux[6][i])
    flux_cat[14].append(fluxerr[6][i])
    flux_cat[15].append(flux[7][i])
    flux_cat[16].append(fluxerr[7][i])
    flux_cat[17].append(flux[8][i])
    tf1=(0.15*flux[8][i])+fluxerr[8][i]
    flux_cat[18].append(tf1)
    flux_cat[19].append(flux[9][i])
    tf2=(0.15*flux[9][i])+fluxerr[9][i]
    flux_cat[20].append(tf2)

ar = np.array(flux_cat)
at = np.transpose(ar)
df = pd.DataFrame(data=at, index=None, columns=['id','F233','E233','F236','E236','F238','E238','F240','E240','F202','E202','F203','E203','F205','E205','F269','E269','F18','E18','F19','E19'])
df['id'] = df['id'].astype('int64')
path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/test_cat_extra.cat'
df.to_csv(path_or_buf=path, sep=' ', index=False)
path = "sed -i '' '/id/s/^/# /g' ../softwares/eazy/inputs/test_cat_extra.cat"
os.system(path)

0

In [7]:
#fill short_table some more with positions, aperture and upper limit/flux info
id_temp = [1, 2, 3, 4, 5, 6, 7] #note: 8 corresponds to 101
ra_temp = [53.1280, 53.1716, 53.0212, 53.0647, 53.2462, 53.1925, 53.1064]
dec_temp = [-27.8594, -27.8208, -27.7824, -27.8626, -27.8618, -27.8668, -27.6977]
id2 = np.zeros(ncat)
ra2 = np.zeros(ncat)
dec2 = np.zeros(ncat)
for i in range(len(ra_temp)):
    id2[i] = id_temp[i]
    ra2[i] = ra_temp[i]
    dec2[i] = dec_temp[i]
cols = fits.ColDefs([fits.Column(name='ID', format='I', array=id2),
                     fits.Column(name='RA', format='D', array=ra2),
                     fits.Column(name='DEC', format='D', array=dec2),
                     fits.Column(name='Aperture', format='D', array=aperture),
                     fits.Column(name='Visual_Flag', format='D', array=visual),
                     fits.Column(name='Spitzer_Aper_Flag', format='D', array=spitzer),
                     fits.Column(name='GOODS_b_Usable', format='D', array=usable[0]),
                     fits.Column(name='GOODS_b_RA', format='D', array=fluxra[0]),
                     fits.Column(name='GOODS_b_Dec', format='D', array=fluxdec[0]),
                     fits.Column(name='GOODS_b_FluxAper', format='D', array=flux[0]),
                     fits.Column(name='GOODS_b_FluxErrAper', format='D', array=fluxerr[0]),
                     fits.Column(name='GOODS_v_Usable', format='D', array=usable[1]),
                     fits.Column(name='GOODS_v_RA', format='D', array=fluxra[1]),
                     fits.Column(name='GOODS_v_Dec', format='D', array=fluxdec[1]),
                     fits.Column(name='GOODS_v_FluxAper', format='D', array=flux[1]),
                     fits.Column(name='GOODS_v_FluxErrAper', format='D', array=fluxerr[1]),
                     fits.Column(name='GOODS_i_Usable', format='D', array=usable[2]),
                     fits.Column(name='GOODS_i_RA', format='D', array=fluxra[2]),
                     fits.Column(name='GOODS_i_Dec', format='D', array=fluxdec[2]),
                     fits.Column(name='GOODS_i_FluxAper', format='D', array=flux[2]),
                     fits.Column(name='GOODS_i_FluxErrAper', format='D', array=fluxerr[2]),
                     fits.Column(name='GOODS_z_Usable', format='D', array=usable[3]),
                     fits.Column(name='GOODS_z_RA', format='D', array=fluxra[3]),
                     fits.Column(name='GOODS_z_Dec', format='D', array=fluxdec[3]),
                     fits.Column(name='GOODS_z_FluxAper', format='D', array=flux[3]),
                     fits.Column(name='GOODS_z_FluxErrAper', format='D', array=fluxerr[3]),
                     fits.Column(name='CANDELS_105_Usable', format='D', array=usable[4]),
                     fits.Column(name='CANDELS_105_RA', format='D', array=fluxra[4]),
                     fits.Column(name='CANDELS_105_Dec', format='D', array=fluxdec[4]),
                     fits.Column(name='CANDELS_105_FluxAper', format='D', array=flux[4]),
                     fits.Column(name='CANDELS_105_FluxErrAper', format='D', array=fluxerr[4]),
                     fits.Column(name='CANDELS_125_Usable', format='D', array=usable[5]),
                     fits.Column(name='CANDELS_125_RA', format='D', array=fluxra[5]),
                     fits.Column(name='CANDELS_125_Dec', format='D', array=fluxdec[5]),
                     fits.Column(name='CANDELS_125_FluxAper', format='D', array=flux[5]),
                     fits.Column(name='CANDELS_125_FluxErrAper', format='D', array=fluxerr[5]),
                     fits.Column(name='CANDELS_160_Usable', format='D', array=usable[6]),
                     fits.Column(name='CANDELS_160_RA', format='D', array=fluxra[6]),
                     fits.Column(name='CANDELS_160_Dec', format='D', array=fluxdec[6]),
                     fits.Column(name='CANDELS_160_FluxAper', format='D', array=flux[6]),
                     fits.Column(name='CANDELS_160_FluxErrAper', format='D', array=fluxerr[6]),
                     fits.Column(name='HUGS_Usable', format='D', array=usable[7]),
                     fits.Column(name='HUGS_RA', format='D', array=fluxra[7]),
                     fits.Column(name='HUGS_Dec', format='D', array=fluxdec[7]),
                     fits.Column(name='HUGS_FluxAper', format='D', array=flux[7]),
                     fits.Column(name='HUGS_FluxErrAper', format='D', array=fluxerr[7]),
                     fits.Column(name='Spitzer_ch1_Usable', format='D', array=usable[8]),
                     fits.Column(name='Spitzer_ch1_RA', format='D', array=fluxra[8]),
                     fits.Column(name='Spitzer_ch1_Dec', format='D', array=fluxdec[8]),
                     fits.Column(name='Spitzer_ch1_FluxAper', format='D', array=flux[8]),
                     fits.Column(name='Spitzer_ch1_FluxErrAper', format='D', array=fluxerr[8]),
                     fits.Column(name='Spitzer_ch2_Usable', format='D', array=usable[9]),
                     fits.Column(name='Spitzer_ch2_RA', format='D', array=fluxra[9]),
                     fits.Column(name='Spitzer_ch2_Dec', format='D', array=fluxdec[9]),
                     fits.Column(name='Spitzer_ch2_FluxAper', format='D', array=flux[9]),
                     fits.Column(name='Spitzer_ch2_FluxErrAper', format='D', array=fluxerr[9])])
hdu2 = fits.BinTableHDU.from_columns(cols)
os.system('rm -f ../data/catalogues/chandra_7ms/short_table.fits')
hdu2.writeto('/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/short_table.fits')

## 